In [69]:
#import library
import pandas as pd
import numpy as np
import time
import seaborn as sns

# Prepare and Analyze Data

1. Load Dataset
2. Analyze Dataset
3. Preprocess data (type, null, missing, ...)
4. Feature Engineering

In [70]:
# PATH = "https://drive.google.com/file/d/1it94WLhc-wB_YwvEhEYjhMErecHX2C_6/view?usp=sharing" # Path to your file
df=pd.read_csv('final_house_loan.csv')

In [71]:
display(df.head())
display(df.describe())

,Gender,Age,Income (USD),Income Stability,Property Age,Property Location,Property Price,Loan Sanction Amount (USD)
0,F,19,1641.25,Low,1651.25,Rural,59641.82,21026.420753
1,M,29,1989.71,Low,1990.71,Urban,179858.51,60595.183366
2,F,37,1849.91,Low,1856.91,Rural,117297.62,39181.648002
3,M,65,2735.18,High,2747.18,Rural,354417.72,128497.710865
4,F,62,4741.78,High,4740.78,Urban,82049.80,39386.919336


,Age,Income (USD),Property Age,Property Price,Loan Sanction Amount (USD)
count,47297.000000,47265.000000,47263.000000,4.729700e+04,47297.000000
mean,40.000063,2586.684384,2586.611058,1.350880e+05,46487.229765
std,16.086128,1558.768809,1558.842286,9.457875e+04,32549.905634
min,18.000000,372.700000,370.700000,7.859620e+03,254.586578
25%,24.000000,1653.740000,1652.820000,6.250408e+04,21782.822159
50%,40.000000,2245.480000,2244.810000,1.130936e+05,38822.132402
75%,55.000000,3128.560000,3128.380000,1.819546e+05,62612.236905
max,65.000000,54662.750000,54647.750000,1.077967e+06,366131.165218


In [72]:
len(df)

47297

In [73]:
df.isnull().sum()

Gender                         0
Age                            0
Income (USD)                  32
Income Stability              12
Property Age                  34
Property Location              3
Property Price                 0
Loan Sanction Amount (USD)     0
dtype: int64

In [74]:
df = df.dropna()

In [75]:
from sklearn.preprocessing import LabelEncoder

# Khởi tạo LabelEncoder
label_encoder = LabelEncoder()

# Chuyển đổi giá trị của cột 'gender' thành số
df['Gender_encoded'] = label_encoder.fit_transform(df['Gender'])
df['Income_Stability_encoded'] = label_encoder.fit_transform(df['Income Stability'])
df['Property_Location_encoded'] = label_encoder.fit_transform(df['Property Location'])

# Xóa các cột phân loại gốc
df = df.drop(columns=['Gender', 'Income Stability', 'Property Location'])


In [76]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [77]:
def prepare_X_y(df):
    # Split data into X and y. Return two dataframes
    X = df.drop('Loan Sanction Amount (USD)', axis=1)
    y = df[['Loan Sanction Amount (USD)']]
    return X, y

X, y = prepare_X_y(df)

In [78]:
def split_train_test(X, y, train_size=0.7):
    trainX, testX ,trainY, testY = train_test_split(X, y, train_size=train_size, random_state=42)
    
    print('Training:' + str(trainX.shape))
    print('Test:' + str(testX.shape))

    return trainX, testX ,trainY, testY

In [79]:
TRAIN_SIZE = 0.7

trainX, testX ,trainY, testY = split_train_test(X, y, train_size=TRAIN_SIZE)

Training:(33075, 7)
Test:(14176, 7)


In [80]:
from sklearn.linear_model import LinearRegression


def build_linear_model(X, y):
    model = LinearRegression()
    model.fit(X, y)
    return model

model = build_linear_model(trainX, trainY)
# Compare on training dataset
pred = model.predict(trainX)   
print("mean absolute error of linear model on train set ", mean_absolute_error(y_pred=pred, y_true=trainY) )
pred = model.predict(testX)   
print("mean absolute error of linear model on test set ", mean_absolute_error(y_pred=pred, y_true=testY) )

print(model.coef_) # print coefficient
print()
print(model.intercept_) # print intercept_

mean absolute error of linear model on train set  148.23672501527736
mean absolute error of linear model on test set  148.78528830546588
[[-7.95663925e+01  5.75232747e-01  4.52094309e-01  3.36212559e-01
  -2.37213059e+02 -1.11589675e+04 -6.54277012e+01]]

[11925.53374574]


When the data feature does not conform to a linear function, a linear regression cannot be applied directly to the original data. Then, there are many possibilities that the data feature conforms to the polynomial function. Scikit-Learn supports converting data features to polynomials through ``PolynomialFeatures``.

$$
y = a_0 + a_1 x + a_2 x^2 + a_3 x^3 + \cdots
$$

The formula above uses the transformation of the value $x$ from one dimension to the other, with the aim of being able to use linear regression to find complex relationships between $x$ and $y$.